## UNIVERSIDAD CENTRAL DEL ECUADOR
### Facultad de Ingeniería y Ciencias Aplicadas
### Criptografía y Seguridad de la Información
**Fecha de entrega:** 10-noviembre-2025

**Grupo 2**

**Integrantes:**


*   Andino Jhon
*   Borja Diego
*   Cajamarca Anthony
*   Cruz Kevin
*   Jami Mateo

# **CRIPTOGRAFÍA ASIMÉTRICA: DSA (Digital Signature Algorithm)**

**DSA**

DSA es un algoritmo asimétrico diseñado para firmas digitales, no para cifrar mensajes.
Fue adoptado por el gobierno de EE. UU. como estándar DSS (Digital Signature Standard).

*Características principales:*

Usa dos claves distintas:
*   Privada: usada para firmar.
*   Pública: usada para verificar la firma.

Basado en la dificultad del logaritmo discreto.

Asegura autenticidad e integridad del mensaje (garantiza que nadie lo modificó).

*Funcionamiento general*
1.   El emisor genera un par de claves (pública y privada).
2.   Firma digitalmente un mensaje con su clave privada.
3.   El receptor verifica la firma usando la clave pública.
4.   Si la verificación es correcta, el mensaje no fue alterado y proviene del emisor.



In [22]:
!pip install pycryptodome

from Crypto.PublicKey import DSA
from Crypto.Signature import DSS
from Crypto.Hash import SHA256
import copy

In [10]:
# --- Generacion de claves ---
key = DSA.generate(2048)  # Genera clave privada (2048 bits)
public_key = key.publickey()  # Deriva la clave pública

In [11]:
mensaje = b"ingeniero geovanny es chevere xD"

In [12]:
# --- Firma Digital ---
hash_obj = SHA256.new(mensaje)
signer = DSS.new(key, 'fips-186-3')  # Estándar DSA
firma = signer.sign(hash_obj)
print("🖋️ Firma digital generada:", firma)

🖋️ Firma digital generada: b'N\xaf\xe7PU\xf5\xaf\x91]\xf9\xb42{\xbf?Un$\x0c\xf39D\xa1\xa2\x8f\x08\xc6\xf0E\xbc#\xf9\x94!\xf3\xff\x86\x08\x19C\xb1\xcd\x06\xbc/u\xedS\xe9\xa2\xf8\xe203\x97n'


In [19]:
# --- Definicion de la funcion verify_signature para los casos de prueba ---
def verify_signature(pub_key, msg, sig):
    h = SHA256.new(msg)
    verifier_func = DSS.new(pub_key, 'fips-186-3')
    try:
        verifier_func.verify(h, sig)
        print("✅ La firma es válida. El mensaje es auténtico.")
    except ValueError:
        print("❌ Firma inválida o mensaje alterado.")

In [23]:
# --- Verificacion
verifier = DSS.new(public_key, 'fips-186-3')
try:
    verifier.verify(hash_obj, firma)
    print("✅ La firma es válida. El mensaje es auténtico.")
except ValueError:
    print("❌ Firma inválida o mensaje alterado.")

✅ La firma es válida. El mensaje es auténtico.


**Casos de pruebas para que su resultado sea erroneo**

In [29]:
# -----------------------------
# Caso 1: modificar el mensaje -> integridad rota
# -----------------------------
print("--- Caso 1: mensaje modificado ---")

mensaje_modificado = b"ingeniero geovanny es chevere xDDD1" # Creamos un mensaje modificado para este caso
verify_signature(public_key, mensaje_modificado, firma)
# ❌ Firma inválida, por qué: la firma cubre el hash del mensaje original; si el mensaje cambia, el hash cambia.

--- Caso 1: mensaje modificado ---
❌ Firma inválida o mensaje alterado.


In [30]:
# -----------------------------
# Caso 2: alterar bytes de la firma -> firma corrupta
# -----------------------------
print("--- Caso 2: firma alterada (un byte cambiado) ---")

sig_list = bytearray(firma) # hacer copia mutable y cambiar un byte
sig_list[0] = (sig_list[0] + 1) % 256  # cambiar un byte (pequeña corrupción)
corrupted_signature = bytes(sig_list)
verify_signature(public_key, mensaje, corrupted_signature)
# ❌ Firma inválida, por qué: la firma es una estructura matemática; cambiar un byte típicamente invalida la verificación.

--- Caso 2: firma alterada (un byte cambiado) ---
❌ Firma inválida o mensaje alterado.


In [31]:
# -----------------------------
# Caso 3: verificar con otra clave pública -> clave no corresponde
# -----------------------------
print("--- Caso 3: verificar con clave pública distinta ---")
other_private = DSA.generate(2048)
other_public = other_private.publickey()
verify_signature(other_public, mensaje, firma)
# ❌ Firma inválida, por qué: la firma fue creada con la clave privada original; otra clave pública no puede verificarla.

--- Caso 3: verificar con clave pública distinta ---
❌ Firma inválida o mensaje alterado.


***`Resumen:`***

Genera un par de claves DSA (privada y pública).

Crea un hash SHA-256 del mensaje (necesario para firmar).

Firma el hash con la clave privada.

Verifica la firma con la clave pública.

## Bibliografia

[1] National Institute of Standards and Technology (NIST), FIPS PUB 186-4: Digital Signature Standard (DSS), Jul. 2013 (Revised Feb. 2024).

[2] Google, "Colaboratory," Google. [Online]. Available: https://colab.research.google.com/. [Accessed: 30-oct-2025].